# Script Teste

_Script_ que define a classe `teste_cepespdata`

In [1]:
import pandas as pd
import numpy as np
import requests
import os

## Função de Testes Específicos

In [2]:
## Votos Brancos
def teste_votos_brancos_nulos(banco):
        brancos = sum(banco.numero_candidato == 95) > 0
        nulos = sum(banco.numero_candidato == 96) > 0
        return {'brancos':brancos, 'nulos':nulos}

## Votos Legenda
# Proporção de votos legenda
def teste_voto_legenda(banco):
    numero_partidos = range(10,91) # partidos entre 10 e 90 inclusivo

    x = [k for k in banco.numero_candidato if len(str(k)) == 2]
    prop_votos_legenda = sum(np.in1d(x, numero_partidos)) / len(banco.numero_candidato)

    return {'votos_legenda': prop_votos_legenda}

## N de Cidades
def teste_cidades(banco):
    n_unique_ibge = len(set(banco.cod_mun_ibge))
    n_unique_tse = len(set(banco.cod_mun_tse))

    return {'ibge': n_unique_ibge, 'tse': n_unique_tse}

## CPF e Títulos
# Proporção de CPF e Títulos com número correto
def teste_cpf_titulos(banco):
    len_titulo = [(lambda x: len(str(x)))(x) for x in banco.num_titulo_eleitoral_candidato]
    len_cpf = [(lambda x: len(str(x)))(x) for x in banco.cpf_candidato]
    prop_titulo = sum(np.array(len_titulo) == 12) / len(len_titulo)
    prop_cpf    = sum(np.array(len_cpf) == 11) / len(len_cpf)
    return {'prop_titulo': prop_titulo, 'prop_cpf': prop_cpf}

In [3]:
class teste_cepespdata:
    
    ## Construtor
    def __init__(self, ano, cargo, agregacao_politica = 2, agregacao_regional = 0, test = False):
        
        if test:
            self.u0 = "http://test.cepesp.io/api/consulta/"
        else:
            self.u0 = 'http://cepesp.io/api/consulta/'
            
        self.ano = str(ano)
        self.cargo = str(cargo)
        self.agregacao_politica = str(agregacao_politica)
        self.agregacao_regional = str(agregacao_regional)
        self.log = {'ano':self.ano,
                    'cargo':self.cargo,
                    'agregacao_politica': self.agregacao_politica,
                    'agregacao_regional': self.agregacao_regional,
                    'banco_candidatos':{},
                    'banco_eleicoes':{}}
    
    ## Download dos Bancos
    def download_eleicoes(self):
        colunas = 'c[]=ANO_ELEICAO&c[]=NUM_TURNO&c[]=COD_MUN_TSE&c[]=COD_MUN_IBGE&c[]=NOME_MUNICIPIO&c[]=UF&c[]=CODIGO_CARGO&c[]=DESCRICAO_CARGO&c[]=NOME_CANDIDATO&c[]=NUMERO_CANDIDATO&c[]=CPF_CANDIDATO&c[]=DES_SITUACAO_CANDIDATURA&c[]=NUMERO_PARTIDO&c[]=SIGLA_PARTIDO&c[]=NUM_TITULO_ELEITORAL_CANDIDATO&c[]=QTDE_VOTOS'
        u0 = self.u0 + "/tse?format=csv&cargo=" + \
                self.cargo + "&anos[]=" + \
                self.ano+"&agregacao_regional=" + \
                self.agregacao_regional+"&agregacao_politica=" + \
                self.agregacao_politica+"&brancos=1&nulos=1&" +\
                colunas
        
        print(u0)
        eleicoes_df = pd.read_csv(u0, sep = ",", dtype = {'num_titulo_eleitoral_candidato': str,
                                                          'cpf_candidato': str})
        self.banco_eleicoes = eleicoes_df
    
    ## Testes
    def download_candidatos(self):
        u0 = self.u0 + "candidatos?format=csv&cargo="+self.cargo+"&anos[]="+self.ano
        candidatos_df = pd.read_csv(u0, sep = ',')
        self.banco_candidatos = candidatos_df
        
    def teste_eleicoes(self):
        b1 = 'banco_eleicoes'
        self.log[b1] = {}
        self.log[b1]['brancos_nulos'] = teste_votos_brancos_nulos(self.banco_eleicoes)
        self.log[b1]['cpf_titulo'] = teste_cpf_titulos(self.banco_eleicoes)
        self.log[b1]['votos_legenda'] = teste_voto_legenda(self.banco_eleicoes)
        if self.agregacao_regional == 6: # Teste n de cidades apenas se agreg. reg. for municipal
            self.log[b1]['n_cidades'] = teste_cidades(self.banco_eleicoes)

                
    def teste(self):
        print("Starting test...\nyear: " + self.ano + "\nposition: " + self.cargo)
        
        self.log['banco_candidatos'] = self.teste_eleicoes()
        
    def __str__(self):
        texto = '''
ANO: {}, CARGO: {}

Branco: {}
Nulos: {}

Proporção de CPF com 11 dígitos: {:.2f}
Proporção de títulos com 12 dígitos: {:.2f}

Proporção de votos em legenda: {:.2f}

'''
        banco_candidatos = self.log['banco_eleicoes']
        print_out = texto.format(self.log['ano'], 
                                 self.log['cargo'],
                                 banco_candidatos['brancos_nulos']['brancos'],
                                 banco_candidatos['brancos_nulos']['nulos'],
                                 banco_candidatos['cpf_titulo']['prop_cpf'],
                                 banco_candidatos['cpf_titulo']['prop_titulo'],
                                 banco_candidatos['votos_legenda']['votos_legenda'])
        return print_out

In [8]:
x = teste_cepespdata(2014,1, agregacao_regional=6)

In [9]:
x.download_eleicoes()

http://cepesp.io/api/consulta//tse?format=csv&cargo=6&anos[]=2014&agregacao_regional=0&agregacao_politica=2&brancos=1&nulos=1&c[]=ANO_ELEICAO&c[]=NUM_TURNO&c[]=COD_MUN_TSE&c[]=COD_MUN_IBGE&c[]=NOME_MUNICIPIO&c[]=UF&c[]=CODIGO_CARGO&c[]=DESCRICAO_CARGO&c[]=NOME_CANDIDATO&c[]=NUMERO_CANDIDATO&c[]=CPF_CANDIDATO&c[]=DES_SITUACAO_CANDIDATURA&c[]=NUMERO_PARTIDO&c[]=SIGLA_PARTIDO&c[]=NUM_TITULO_ELEITORAL_CANDIDATO&c[]=QTDE_VOTOS


In [10]:
x.teste()

Starting test...
year: 2014
position: 6


In [11]:
print(x)


ANO: 2014, CARGO: 6

Branco: True
Nulos: True

Proporção de CPF com 11 dígitos: 0.88
Proporção de títulos com 12 dígitos: 0.88

Proporção de votos em legenda: 0.12


